In [1]:
import json
from tqdm import tqdm

In [2]:
big = "memoryalpha"
small = "stexpanded"

mapping_file_big = "./data/triples_v2/" + big + "_mapping.json"
mapping_file_small = "./data/triples_v2/" + small + "_mapping.json"
exact_match_file = "./data/exact_match/" + big + "-" + small + ".json"

# communities_big_file = "./data/one_step/" + big + ".csv"
# communities_small_file = "./data/one_step/" + small + ".csv"

communities_big_file = "./results/communities_bigclam/" + big + "_10000.txt"
communities_small_file = "./results/communities_bigclam/" + small + "_10000.txt"

In [3]:
with open(exact_match_file) as emf, open(mapping_file_big) as mfb, open(mapping_file_small) as mfs:
    exact_match = json.load(emf)
    mapping_big = json.load(mfb)
    mapping_small = json.load(mfs)

    big_id_dict = {key: value for key, value in mapping_big.items()}
    small_id_dict = {key: value for key, value in mapping_small.items()}
    
    formatted_json = []
    for pair in exact_match:
        big_id = big_id_dict.get(pair[0])
        small_id = small_id_dict.get(pair[1])
        if big_id and small_id:
            formatted_json.append([big_id, small_id])

    # with open('formatted.json', 'w') as outfile:
        # json.dump(formatted_json, outfile, indent=4)
    
# print(formatted_json)

In [4]:
# in_exact_match_big_file = "./data/one_step_in_exact_match/" + big + ".csv"
# in_exact_match_small_file = "./data/one_step_in_exact_match/" + small + ".csv"
# 
# big_id_keep = set()
# small_id_keep = set()
# 
# for pair in formatted_json:
#     big_id_keep.add(pair[0])
#     small_id_keep.add(pair[1])
# 
# with open(communities_big_file, "r") as cbf, open(in_exact_match_big_file, "w") as output:
#     for line in cbf:
#         numbers = [int(num) for num in line.strip().split('\t')]
#         for number in numbers:
#             if number in big_id_keep:
#                 output.write(str(number) + ";")
#         output.write("\n")
# 
# with open(communities_small_file, "r") as csf, open(in_exact_match_small_file, "w") as output:
#     for line in csf:
#         numbers = [int(num) for num in line.strip().split('\t')]
#         for number in numbers:
#             if number in small_id_keep:
#                 output.write(str(number) + ";")
#         output.write("\n")

In [13]:
small_communities = []
big_communities = []

len_small_communities = []
len_big_communities = []

with (open(communities_big_file) as cbf, open(communities_small_file) as csf):
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split("\t")}
        small_communities.append(numbers_set)
        len_small_communities.append(len(numbers_set))

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split("\t")}
        big_communities.append(numbers_set)
        len_big_communities.append(len(numbers_set))
        
    # print(len_small_communities)

    exact_match_count_dict = dict()
    # for id_big, id_small in tqdm(formatted_json):
    for id_big, id_small in formatted_json:
        # print(id_big, id_small)
        indexes_big = [index for index, s in enumerate(big_communities) if id_big in s]
        indexes_small = [index for index, s in enumerate(small_communities) if id_small in s]
        # print(indexes_big)
        # print(indexes_small)
        for i_big in indexes_big:
            for i_small in indexes_small:
                if (i_big, i_small) in exact_match_count_dict:
                    exact_match_count_dict[(i_big, i_small)] += 1
                else:
                    exact_match_count_dict[(i_big, i_small)] = 1
                    
    # print(exact_match_count_dict.get((0,0)))

    exact_match_percentage_dict = dict()
    for key, value in exact_match_count_dict.items():
        # print(key, value)
        # print(key[0], key[1])
        # print(len_small_communities[key[1]])
        # print(value / len_small_communities[key[1]])
        # exact_match_percentage_dict[key] = value
        # exact_match_percentage_dict[key] = value / len_small_communities[key[1]]
        # exact_match_percentage_dict[key] = value / len_big_communities[key[0]]
        exact_match_percentage_dict[key] = (value
                                            / (len_big_communities[key[0]] * len_small_communities[key[1]])
                                            * (1 - (abs(len_big_communities[key[0]] - len_small_communities[key[1]]))
                                            / max(len_big_communities[key[0]], len_small_communities[key[1]]))
                                            )
        # exact_match_percentage_dict[key] = value / (len_big_communities[key[0]] + len_small_communities[key[1]] - value)

    exact_match_best_dict = dict()
    for key, value in exact_match_percentage_dict.items():
        if key[0] in exact_match_best_dict:
            if value > exact_match_best_dict[key[0]][1]:
                exact_match_best_dict[key[0]] = (key[1], value)
        else:
            exact_match_best_dict[key[0]] = (key[1], value)
            
print(exact_match_best_dict)

{7: (0, 2.1488544238538867e-05), 41: (4, 0.00013287197231833908), 367: (191, 0.0010170118343195266), 1070: (137, 0.0015608740894901144), 0: (0, 4.531832105649103e-06), 229: (263, 0.000234375), 1: (0, 4.482495412092059e-06), 8: (3, 1.5993848519800075e-05), 9: (4, 3.501277966457757e-05), 12: (2, 1.8003161355133963e-05), 1471: (5742, 0.001040582726326743), 6382: (1895, 0.010204081632653062), 19: (2, 3.536271913253389e-05), 26: (7, 6.940741839107293e-05), 42: (14, 7.673360394717658e-05), 85: (998, 0.00013548070246744226), 86: (13, 0.00019290123456790125), 88: (998, 0.00016336437256412047), 98: (2849, 0.0001811375437749065), 117: (14, 0.00017185083347654237), 127: (13, 0.00014467592592592594), 130: (263, 0.00018140589569161), 132: (998, 0.00016656751933372986), 160: (998, 0.0001939058171745152), 174: (18, 0.00020824656393169514), 178: (405, 0.00014768431001890365), 190: (998, 0.0002312062359624786), 196: (998, 0.00016976007243096425), 202: (998, 0.0002612244897959183), 252: (998, 0.00027005

In [6]:
exact_match_best_dict.get(7331)

(4639, 0.38461538461538464)

In [8]:
exact_match_best_dict.get(7331)

(4639, 0.3333333333333333)

In [20]:
print(big_communities[7331])
print(small_communities[4639])

In [13]:
print(str(exact_match_count_dict))

In [14]:
exact_match_count_dict[(0,14)]

In [6]:
len_small_communities[3701]

13